In [1]:
import os
import os.path as path
import gc
import re
import math
import json

In [2]:
import numpy as np
import pandas as pd

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
pd.set_option('display.max_row', 50)
pd.set_option('display.max_columns', 100)

In [6]:
# 디렉토리 기본 경로 설정
DIR_PATH = path.join('..', 'data')
DIR_SAVE_PATH = path.join('..', 'data', 'output')

print(DIR_PATH)
print(DIR_SAVE_PATH)

..\data
..\data\output


In [16]:
# bean_raw_data.csv
user_read = pd.read_csv(path.join(DIR_PATH, 'user_propensity_data.csv'), low_memory=False)

print(user_read.shape)
user_read.columns = ['timestamp', 'phone', 'cafe', 'preference', 'drink', 'dessert', 'fruit', 'aroma']
user_read.head()

(105, 8)


,timestamp,phone,cafe,preference,drink,dessert,fruit,aroma
0,2023. 3. 24 오후 2:59:41,010-9491-7668,스타벅스 Starbucks,향이나 맛이 다른 커피보다 잘 느껴져요,"☕에스프레소 또는 아메리카노, 콜드브루",진하고 달달한 초코 케이크,"체리, 복숭아",허브의 마음이 진정되는 향기
1,2023. 3. 24 오후 3:03:33,010-2840-2336,스타벅스 Starbucks,다른 커피를 마실때보다 목넘김이 좋아요,"🍯카페모카 또는 바닐라라떼, 카라멜 마키아또 (달달해요)",진하고 달달한 초코 케이크,"사과, 배",꿀이나 설탕같은 달달한 향기
2,2023. 3. 24 오후 3:03:50,010-9231-9670,투썸플레이스 A TWOSOME PLACE,향이나 맛이 다른 커피보다 잘 느껴져요,"☕에스프레소 또는 아메리카노, 콜드브루",상큼하고 개운한 과일 타르트,"크랜베리, 블루베리",허브의 마음이 진정되는 향기
3,2023. 3. 24 오후 3:04:09,010-9027-0157,이디야 커피 EDIYA COFFEE,다른 커피를 마실때보다 목넘김이 좋아요,"☕에스프레소 또는 아메리카노, 콜드브루",진하고 달달한 초코 케이크,"레몬, 오렌지, 귤",나무의 우디한 향기
4,2023. 3. 24 오후 3:04:13,010-4404-9238,스타벅스 Starbucks,설탕이나 초콜릿 냄새라던가 맛이 선명해요,"🍯카페모카 또는 바닐라라떼, 카라멜 마키아또 (달달해요)",상큼하고 개운한 과일 타르트,"사과, 배",꽃의 신선한 향기


In [17]:
user_data = user_read.iloc[:, 2:].copy()
user_data

,cafe,preference,drink,dessert,fruit,aroma
0,스타벅스 Starbucks,향이나 맛이 다른 커피보다 잘 느껴져요,"☕에스프레소 또는 아메리카노, 콜드브루",진하고 달달한 초코 케이크,"체리, 복숭아",허브의 마음이 진정되는 향기
1,스타벅스 Starbucks,다른 커피를 마실때보다 목넘김이 좋아요,"🍯카페모카 또는 바닐라라떼, 카라멜 마키아또 (달달해요)",진하고 달달한 초코 케이크,"사과, 배",꿀이나 설탕같은 달달한 향기
2,투썸플레이스 A TWOSOME PLACE,향이나 맛이 다른 커피보다 잘 느껴져요,"☕에스프레소 또는 아메리카노, 콜드브루",상큼하고 개운한 과일 타르트,"크랜베리, 블루베리",허브의 마음이 진정되는 향기
3,이디야 커피 EDIYA COFFEE,다른 커피를 마실때보다 목넘김이 좋아요,"☕에스프레소 또는 아메리카노, 콜드브루",진하고 달달한 초코 케이크,"레몬, 오렌지, 귤",나무의 우디한 향기
4,스타벅스 Starbucks,설탕이나 초콜릿 냄새라던가 맛이 선명해요,"🍯카페모카 또는 바닐라라떼, 카라멜 마키아또 (달달해요)",상큼하고 개운한 과일 타르트,"사과, 배",꽃의 신선한 향기
...,...,...,...,...,...,...
100,컴포즈,커피의 새콤한 맛이 도드라져요,"☕에스프레소 또는 아메리카노, 콜드브루",진하고 달달한 초코 케이크,"레몬, 오렌지, 귤",꽃의 신선한 향기
101,이디야 커피 EDIYA COFFEE,커피 특유의 씁쓸한 맛이 잘 나타나요,"☕에스프레소 또는 아메리카노, 콜드브루",상큼하고 개운한 과일 타르트,"레몬, 오렌지, 귤",나무의 우디한 향기
102,카페 파스쿠치 CAFFÉ PASCUCCI,다른 커피를 마실때보다 목넘김이 좋아요,"☕에스프레소 또는 아메리카노, 콜드브루",진하고 달달한 초코 케이크,"크랜베리, 블루베리",나무의 우디한 향기
103,투썸플레이스 A TWOSOME PLACE,커피의 새콤한 맛이 도드라져요,"☕에스프레소 또는 아메리카노, 콜드브루",담백하고 고소한이 두드러지는 호두파이,"망고, 바나나, 파인애플",허브의 마음이 진정되는 향기
